In [1]:
import os
import re
import csv
import torch
import tifffile
import numpy as np
from scipy.ndimage import zoom
# from scipy.ndimage import binary_dilation

In [2]:
import torch

# accelerate using GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [24]:
# Conversion table for image names

# identify channel names in registered image filenames
conversion_table1 = {
    "Time0_channel1": "SF3A66", "Time1_channel1": "Fibrillarin", "Time1_channel0": "H3K4me3",
    "Time2_channel2": "H3K27ac", "Time2_channel1": "HP1a", "Time2_channel0": "H3K27me3",
    "Time3_channel2": "CDK9", "Time3_channel1": "H2A_Z", "Time3_channel0": "H3K4me1",
    "Time4_channel2": "mH2A1", "Time4_channel0": "PolII", "Time5_channel2": "Matrin3",
    "Time6_channel2": "NPM1", "Time6_channel3": "DAPI"
}

# identify channel names in extracted nuclear image filenames
conversion_table2 = {
    "t0_c1": "SF3A66", "t1_c1": "Fibrillarin", "t1_c0": "H3K4me3",
    "t2_c2": "H3K27ac", "t2_c1": "HP1a", "t2_c0": "H3K27me3",
    "t3_c2": "CDK9", "t3_c1": "H2A_Z", "t3_c0": "H3K4me1",
    "t4_c2": "mH2A1", "t4_c0": "PolII", "t5_c2": "Matrin3", "t6_c2": "NPM1", "t6_c3": "DAPI"
}

In [42]:
zoom_factor = 1 / 5.8824   # 5.8824 is the z scaling factor applied before for converting to isotropic voxel

def process_images(base_folder, mask_folder, results_folder, output_folder, conversion_table1, conversion_table2, zoom_factor):

    # Name output file for measurements of all images in the folder
    overall_output_file = os.path.join(results_folder, 'astrocyte_nuclear_intensity_all.csv')
    
    pattern = re.compile(r'cell(\d+)')         
    all_proteins = list(conversion_table1.values())
    
    overall_intensity_data = []  # Store all results for aggregated CSV

    for i in range(0, 41):  # 41 fields of view

        # read single-nucleus images stored in individual folders for each field of view
        folder_name = f"z{i}"   
        print("Processing folder:", folder_name)
        image_folder = os.path.join(base_folder, folder_name)
        
        mask_image_path = os.path.join(mask_folder, f"z{i}.tif")

        # Load and transfer mask to GPU
        mask_image = tifffile.imread(mask_image_path).astype(np.uint16)  # Convert to uint16
        mask_image = torch.tensor(mask_image, dtype=torch.float32, device=device)

        # rescales mask in z-axis to match registered image
        mask_rescaled = torch.tensor(zoom(mask_image.cpu().numpy(), (zoom_factor, 1, 1), order=0), device=device)

        intensity_data = {}

        for key, protein_name in conversion_table1.items():      # loop through proteins
            # print(protein_name)

            # load registered images
            img_tif = os.path.join("./Bigstream_notebook/", f"Aligned_zstack{i}_{key}.tiff")
            image_data = tifffile.imread(img_tif).astype(np.uint16)
            image_tensor = torch.tensor(image_data, dtype=torch.float32, device=device)

            # read cell numbers from extracted nuclear images
            key2 = next(key for key, value in conversion_table2.items() if value == protein_name)
            for filename in os.listdir(image_folder):
                if key2 in filename and filename.endswith('.tiff'):
                    
                    cell_number = int(filename.split('cell')[1].split('.tiff')[0])
                    # print(cell_number)

                    # find voxels in the mask with the same value as the current cell number -- mask belongs to this cell 
                    # binary (0 = background, 255 = mask)
                    cell_mask = (mask_rescaled == cell_number).to(torch.uint8) * 255

                    # Each protein loop visits the same cell multiple times. 
                    # Only create the entry once, then fill in protein intensities.
                    if (folder_name, cell_number) not in intensity_data:
                        intensity_data[(folder_name, cell_number)] = {protein: None for protein in all_proteins}
                        intensity_data[(folder_name, cell_number)]['Folder Name'] = folder_name
                        intensity_data[(folder_name, cell_number)]['Cell Number'] = cell_number

                    # finds all voxels belonging to the cell, calculate the mean intensity in registered image.
                    if torch.any(cell_mask):
                        masked_voxels = image_tensor[cell_mask == 255]
                        mean_gray_level = torch.mean(masked_voxels).item()
                        # print(mean_gray_level)
                        intensity_data[(folder_name, cell_number)][protein_name] = mean_gray_level
                        
        # Save per field-of-view CSV
        z_output_file = os.path.join(output_folder, f'intensity_{folder_name}.csv')
        with open(z_output_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            header = ['Folder Name', 'Cell Number'] + all_proteins
            writer.writerow(header)

            for cell_data in intensity_data.values():
                row = [cell_data.get(col, None) for col in header]
                writer.writerow(row)
        
        # print(f"Saved per-folder CSV: {z_output_file}")

        # Append to overall dataset
        overall_intensity_data.extend(intensity_data.values())

    # Save the aggregated CSV for all fields of view
    with open(overall_output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        header = ['Folder Name', 'Cell Number'] + all_proteins
        writer.writerow(header)

        for cell_data in overall_intensity_data:
            row = [cell_data.get(col, None) for col in header]
            writer.writerow(row)

if __name__ == "__main__":
    base_folder = "./segmented_glials/"         # Directory right above images. Need to process each image folder separately.
    mask_folder = "./CP_masks4/"                # where Cellpose masks are stored
    results_folder = r"E:\Neuron_17Ab_analysis\cell_measurements"      # measurements for each image as a separate file
    output_folder = r"E:\Neuron_17Ab_analysis\cell_measurements\astrocyte_protein_intensity_z"  # directory to save file combining results from all images.

process_images(base_folder, mask_folder, results_folder, output_folder, conversion_table1, conversion_table2, zoom_factor)

Processing folder: z0
Processing folder: z1
Processing folder: z2
Processing folder: z3
Processing folder: z4
Processing folder: z5
Processing folder: z6
Processing folder: z7
Processing folder: z8
Processing folder: z9
Processing folder: z10
Processing folder: z11
Processing folder: z12
Processing folder: z13
Processing folder: z14
Processing folder: z15
Processing folder: z16
Processing folder: z17
Processing folder: z18
Processing folder: z19
Processing folder: z20
Processing folder: z21
Processing folder: z22
Processing folder: z23
Processing folder: z24
Processing folder: z25
Processing folder: z26
Processing folder: z27
Processing folder: z28
Processing folder: z29
Processing folder: z30
Processing folder: z31
Processing folder: z32
Processing folder: z33
Processing folder: z34
Processing folder: z35
Processing folder: z36
Processing folder: z37
Processing folder: z38
Processing folder: z39
Processing folder: z40
